# CNN from scratch

This notebook is intended for CIFAR-10 object recognition using CNN from scratch.

First, some imports.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Activation, add
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import RMSprop
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as ktf
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook

Then, read the data, and normalize them.

In [ ]:
# read data saved in Keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# check data size
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# normalize them to 0 mean and unit variance
data = ImageDataGenerator(samplewise_center=True,
                          samplewise_std_normalization=True,
                          featurewise_center=False,
                          featurewise_std_normalization=False)
data.fit(x_train)

# SimpleCNN

Here we use a simple CNN model and train it from scratch.

In [ ]:
# model architecture
# [batchnorm-Conv-Conv-maxpool]x2 - [dense]x2 - [softmax]
simple_CNN = Sequential()
simple_CNN.add(BatchNormalization(input_shape=(32, 32, 3)))
simple_CNN.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (16,16,32)

simple_CNN.add(BatchNormalization())
simple_CNN.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
simple_CNN.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
simple_CNN.add(MaxPooling2D((2, 2))) # (8,8,64)

simple_CNN.add(Flatten())
simple_CNN.add(Dense(2048, activation='relu'))
simple_CNN.add(Dropout(0.45))
simple_CNN.add(Dense(10, activation='softmax'))

# set loss and optimizer
rmsprop = RMSprop(lr=0.001, decay=0.99)
simple_CNN.compile(loss='sparse_categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

# train the model
checkpoint = ModelCheckpoint('../models/simpleCNN_{epoch:02d}-{loss:.4f}.h5',
                             monitor='loss',
                             save_best_only=True)
earlystop = EarlyStopping(min_delta=0.0001, patience=3)

simple_CNN.fit_generator(generator=data.flow(x_train, y_train, batch_size=64),
                         steps_per_epoch=len(x_train)/64,
                         initial_epoch=0,
                         epochs=10,
                         callbacks=[],
                         validation_data=(x_test,y_test.reshape(-1,1)))

# Finetuning

# Create submission